In [2]:
import json
import pandas as pd

word_list = []
with open(r"word_list\spanish\es_wordlist.json") as file:
    json_data = json.load(file)
    for d in json_data.items():
        word_list.append([d[0], d[1]])

In [3]:
df = pd.DataFrame(word_list)
df.columns = ["word", "count"]

# only lowercase word
df["word"] = df["word"].apply(lambda x: str(x).lower())
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)
df = df[df["word"].str.isalpha()]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,count,word_len
0,a,22256821,1
1,aa,2017,2
2,aaa,1331,3
3,aaaa,21,4
4,aaaaa,3,5


In [4]:
df.dtypes

word        object
count        int64
word_len     int64
dtype: object

In [5]:
df.describe()

,count,word_len
count,2.167783e+06,2.167783e+06
mean,4.615537e+02,9.816662e+00
std,7.763210e+04,4.628269e+00
min,1.000000e+00,1.000000e+00
25%,1.000000e+00,7.000000e+00
50%,2.000000e+00,9.000000e+00
75%,5.000000e+00,1.100000e+01
max,7.534259e+07,1.200000e+02


In [6]:
df[df["word_len"] == 7].describe()

,count,word_len
count,2.880210e+05,288021.0
mean,3.380390e+02,7.0
std,9.227966e+03,0.0
min,1.000000e+00,7.0
25%,1.000000e+00,7.0
50%,2.000000e+00,7.0
75%,6.000000e+00,7.0
max,1.625156e+06,7.0


In [7]:
df.query("'libro' in word")

,word,count,word_len
1222753,libro,62042,5


In [8]:
difficulties = {"easy": 1000, "medium": 3000, "hard": 10000}
lengths = [5, 6, 7]


for l in lengths:
    for name, count in difficulties.items():
        df_filtered = (
            df[df["word_len"] == l]
            .sort_values(by=["count"], ascending=False)
            .head(count)
            .reset_index(drop=True)
        )
        with open(rf"vocabs\es\es-len{l}-{name}.json", "w") as f:
            data = df_filtered["word"].sort_values().tolist()
            json.dump(data, f)